In [6]:
import requests
import pandas as pd
from yahoo_fin.stock_info import get_data
from datetime import datetime

class StockDataExtract:
    def __init__(self, ticker, start_date='2023-03-07', end_date='2024-03-07'):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.marketdata_url_template = "https://api.marketdata.app/v1/stocks/candles/D/{ticker}?from={start_date}&to={end_date}"

    def fetch_yahoo_finance_data(self):
        try:
            df = pd.DataFrame(get_data(self.ticker))
            # Convert index to datetime to ensure proper comparison
            df.index = pd.to_datetime(df.index)
            # Filter data based on start_date and end_date
            filtered_df = df.loc[self.start_date:self.end_date]
            filtered_df = filtered_df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "adjclose": "AdjClose", "volume": "Volume"})
            filtered_df['Date'] = filtered_df.index
            filtered_df.reset_index(drop=True, inplace=True)
            return filtered_df
        except Exception as e:
            print(f"Error fetching data from Yahoo Finance: {e}")
            return None

    def fetch_marketdata_api_data(self):
        try:
            url = self.marketdata_url_template.format(ticker=self.ticker, start_date=self.start_date, end_date=self.end_date)
            response = requests.get(url)
            if response.status_code != 200:
                print(f"Error fetching data from MarketData API: HTTP Status Code {response.status_code}")
                return None
            json_data = response.json()
            if 't' not in json_data:
                print(f"Key 't' not found in response. Response content: {response.content}")
                return None
            dates = [datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d') for timestamp in json_data['t']]
            data = {
                "Date": dates,
                "Open_MarketData": json_data.get("o", []),
                "Close_MarketData": json_data.get("c", []),
                "Highest_MarketData": json_data.get("h", []),
                "Lowest_MarketData": json_data.get("l", []),
            }
            df = pd.DataFrame(data)
            df['Date'] = pd.to_datetime(df['Date'])
            return df
        except Exception as e:
            print(f"Exception occurred: {e}")
            return None

    def merge_data(self, df1, df2):
        # Ensure 'Date' is the datetime type for both DataFrames
        df1['Date'] = pd.to_datetime(df1['Date'])
        df2['Date'] = pd.to_datetime(df2['Date'])

        # Merge the dataframes on 'Date'
        merged_df = pd.merge(df1, df2, on='Date', how='outer', suffixes=('', '_MarketData'))
        
        # Drop the MarketData columns if they exist, as we only need them once
        merged_df.drop(columns=['Open_MarketData', 'Close_MarketData', 'Highest_MarketData', 'Lowest_MarketData'], inplace=True, errors='ignore')
        
        # Set 'Date' as index after sorting by it
        merged_df.sort_values('Date', inplace=True)
        merged_df.set_index('Date', inplace=True)
        
        return merged_df

    def get_merged_stock_data(self):
        yahoo_data = self.fetch_yahoo_finance_data()
        marketdata_api_data = self.fetch_marketdata_api_data()
        merged_data = self.merge_data(yahoo_data, marketdata_api_data)
        return merged_data

# Example usage:
ticker = 'AAPL'
stock_extractor = StockDataExtract(ticker)
merged_stock_data = stock_extractor.get_merged_stock_data()

In [7]:
merged_stock_data

,Open,High,Low,Close,AdjClose,Volume,ticker
Date,,,,,,,
2023-03-07,153.699997,154.029999,151.130005,151.600006,150.795105,56182000,AAPL
2023-03-08,152.809998,153.470001,151.830002,152.869995,152.058350,47204800,AAPL
2023-03-09,153.559998,154.539993,150.229996,150.589996,149.790466,53833600,AAPL
2023-03-10,150.210007,150.940002,147.610001,148.500000,147.711548,68572400,AAPL
2023-03-13,147.809998,153.139999,147.699997,150.470001,149.671112,84457100,AAPL
...,...,...,...,...,...,...,...
2024-03-01,179.550003,180.529999,177.380005,179.660004,179.660004,73488000,AAPL
2024-03-04,176.149994,176.899994,173.789993,175.100006,175.100006,81510100,AAPL
2024-03-05,170.759995,172.039993,169.619995,170.119995,170.119995,95132400,AAPL


In [8]:
class StockIndicators:
    def __init__(self, data):
        self.data = data
        # Default periods for each indicator
        self.williams_r_period = 14
        self.ema_period = 14
        self.sto_lookback_period = 14
        self.sto_sma_period = 3
        self.bollinger_bands_period = 20
        self.bollinger_bands_multiplier = 2
        self.atr_period = 14

    def calculate_all_indicators(self):
        """
        Calculate all indicators with their default period values.
        """
        self.calculate_williams_r(self.williams_r_period)
        self.calculate_ema(self.ema_period)
        self.calculate_stochastic_oscillator(self.sto_lookback_period, self.sto_sma_period)
        self.calculate_bollinger_bands(self.bollinger_bands_period, self.bollinger_bands_multiplier)
        self.calculate_atr(self.atr_period)
        self.calculate_adl()
        self.calculate_ichimoku_cloud()

    # Individual indicator calculation methods follow

    def calculate_williams_r(self, n):
        highest_high = self.data['High'].rolling(window=n).max()
        lowest_low = self.data['Low'].rolling(window=n).min()
        self.data['Williams_R'] = (highest_high - self.data['Close']) / (highest_high - lowest_low) * -100

    def calculate_ema(self, period):
        self.data[f'EMA_{period}'] = self.data['Close'].ewm(span=period, adjust=False).mean()

    def calculate_stochastic_oscillator(self, lookback_period, sma_period):
        lowest_low = self.data['Low'].rolling(window=lookback_period).min()
        highest_high = self.data['High'].rolling(window=lookback_period).max()
        self.data['%K'] = ((self.data['Close'] - lowest_low) / (highest_high - lowest_low)) * 100
        self.data['%D'] = self.data['%K'].rolling(window=sma_period).mean()

    def calculate_bollinger_bands(self, period, multiplier):
        self.data['Middle_Band'] = self.data['Close'].rolling(window=period).mean()
        std_dev = self.data['Close'].rolling(window=period).std()
        self.data['Upper_Band'] = self.data['Middle_Band'] + (std_dev * multiplier)
        self.data['Lower_Band'] = self.data['Middle_Band'] - (std_dev * multiplier)

    def calculate_atr(self, period):
        high_low = self.data['High'] - self.data['Low']
        high_close = abs(self.data['High'] - self.data['Close'].shift())
        low_close = abs(self.data['Low'] - self.data['Close'].shift())
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        self.data[f'ATR_{period}'] = tr.rolling(window=period).mean()

    def calculate_adl(self):
        clv = ((self.data['Close'] - self.data['Low']) - (self.data['High'] - self.data['Close'])) / (self.data['High'] - self.data['Low'])
        clv = clv.fillna(0)  # Replace NaN values with 0
        self.data['ADL'] = (clv * self.data['Volume']).cumsum()

    def calculate_ichimoku_cloud(self):
        high_prices = self.data['High']
        low_prices = self.data['Low']
        self.data['Tenkan_Sen'] = (high_prices.rolling(window=9).max() + low_prices.rolling(window=9).min()) / 2
        self.data['Kijun_Sen'] = (high_prices.rolling(window=26).max() + low_prices.rolling(window=26).min()) / 2
        self.data['Senkou_Span_A'] = ((self.data['Tenkan_Sen'] + self.data['Kijun_Sen']) / 2).shift(26)
        self.data['Senkou_Span_B'] = ((high_prices.rolling(window=52).max() + low_prices.rolling(window=52).min()) / 2).shift(26)
        self.data['Chikou_Span'] = self.data['Close'].shift(-26)

In [9]:
df = merged_stock_data
indicators = StockIndicators(df)

# Calculate all indicators
indicators.calculate_all_indicators()

# Access the modified DataFrame with all indicators
modified_df = indicators.data
print(modified_df)

                  Open        High         Low       Close    AdjClose  \
Date                                                                     
2023-03-07  153.699997  154.029999  151.130005  151.600006  150.795105   
2023-03-08  152.809998  153.470001  151.830002  152.869995  152.058350   
2023-03-09  153.559998  154.539993  150.229996  150.589996  149.790466   
2023-03-10  150.210007  150.940002  147.610001  148.500000  147.711548   
2023-03-13  147.809998  153.139999  147.699997  150.470001  149.671112   
...                ...         ...         ...         ...         ...   
2024-03-01  179.550003  180.529999  177.380005  179.660004  179.660004   
2024-03-04  176.149994  176.899994  173.789993  175.100006  175.100006   
2024-03-05  170.759995  172.039993  169.619995  170.119995  170.119995   
2024-03-06  171.059998  171.240005  168.679993  169.119995  169.119995   
2024-03-07  169.149994  170.729996  168.490005  169.000000  169.000000   

              Volume ticker  Williams

In [10]:
modified_df

,Open,High,Low,Close,AdjClose,Volume,ticker,Williams_R,EMA_14,%K,...,Middle_Band,Upper_Band,Lower_Band,ATR_14,ADL,Tenkan_Sen,Kijun_Sen,Senkou_Span_A,Senkou_Span_B,Chikou_Span
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-07,153.699997,154.029999,151.130005,151.600006,150.795105,56182000,AAPL,NaN,151.600006,NaN,...,NaN,NaN,NaN,NaN,-3.797120e+07,NaN,NaN,NaN,NaN,165.559998
2023-03-08,152.809998,153.470001,151.830002,152.869995,152.058350,47204800,AAPL,NaN,151.769338,NaN,...,NaN,NaN,NaN,NaN,-2.530686e+07,NaN,NaN,NaN,NaN,165.210007
2023-03-09,153.559998,154.539993,150.229996,150.589996,149.790466,53833600,AAPL,NaN,151.612092,NaN,...,NaN,NaN,NaN,NaN,-7.014735e+07,NaN,NaN,NaN,NaN,165.229996
2023-03-10,150.210007,150.940002,147.610001,148.500000,147.711548,68572400,AAPL,NaN,151.197147,NaN,...,NaN,NaN,NaN,NaN,-1.020655e+08,NaN,NaN,NaN,NaN,166.470001
2023-03-13,147.809998,153.139999,147.699997,150.470001,149.671112,84457100,AAPL,NaN,151.100194,NaN,...,NaN,NaN,NaN,NaN,-1.005129e+08,NaN,NaN,NaN,NaN,167.630005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01,179.550003,180.529999,177.380005,179.660004,179.660004,73488000,AAPL,-79.805136,182.865406,20.194864,...,184.415501,190.640370,178.190632,2.935000,1.318440e+09,181.209999,186.825005,188.812500,189.894997,NaN
2024-03-04,176.149994,176.899994,173.789993,175.100006,175.100006,81510100,AAPL,-89.452404,181.830020,10.547596,...,183.878001,191.319000,176.437002,3.207142,1.305598e+09,179.414993,184.274994,188.632500,189.894997,NaN
2024-03-05,170.759995,172.039993,169.619995,170.119995,170.119995,95132400,AAPL,-96.857323,180.268683,3.142677,...,183.000001,192.430189,173.569813,3.338572,1.249777e+09,177.329994,180.909996,188.632500,189.894997,NaN
